## Aerial Bombing Operations in World War 2##

This notebook is based on the [kaggle dataset](https://www.kaggle.com/usaf/world-war-ii) of the same name.

World War 2 is a topic that is of great interest to me and I hope to find something interesting regarding the aircraft used and the main theatres of war/areas with most war activity from the USAF's point of view.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt



Let us first import the dataset and store it.

In [31]:
data = pd.read_csv('operations.csv')

We will first explore the dataset to get a overview of the data.

In [32]:
data.shape

(178281, 46)

The dataset has 178281 rows of data across 46 different columns. We will next explore the 46 columns.

There is a wealth of information in this dataset. Let us check if we have any missing or dummy data in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178281 entries, 0 to 178280
Data columns (total 46 columns):
Mission ID                               178281 non-null int64
Mission Date                             178281 non-null object
Theater of Operations                    175123 non-null object
Country                                  126494 non-null object
Air Force                                126444 non-null object
Unit ID                                  50913 non-null object
Aircraft Series                          178165 non-null object
Callsign                                 21 non-null object
Mission Type                             47186 non-null object
Takeoff Base                             3383 non-null object
Takeoff Location                         3338 non-null object
Takeoff Latitude                         2981 non-null object
Takeoff Longitude                        2980 non-null float64
Target ID                                127406 non-null object
Target 

The data seems to be quite dirty and may need extensive cleaning. There are multiple columns with very few rows of data. My initial hunch was to remove them since they have very less data, but I decided to leave them as they are and only use the columns which have a good amount of data. In the data cleaning process my focus would be on cleaning the columns which have enough data to regenerate the missing rows.

In [33]:
data = data.drop('Callsign',axis=1)

Let us check the shape of the data to ensure that the Callsign column has been dropped.

In [34]:
data.shape

(178281, 45)

Let us start our analysis with aircraft series since it has the least amount of missing data.

In [35]:
len(data['Aircraft Series'].unique())

75

75 different types of bomber aircraft are in the dataset.

In [36]:
data['Aircraft Series'].value_counts()

B24              33837
B17              31448
B25              17359
A20              12561
B26              11980
GB17              9124
WELL              6391
HVY               6298
A26               4736
P47               4660
LGT               3673
P51               3623
HAMP              3552
BLEN              3223
WHIT              3205
GB24              3110
P38               3001
P40               2891
B29               2518
MED               2379
A36               1384
OB24              1067
P39                893
SB17               855
LIB                769
OB17               490
SB24               447
STIR               404
F4U                361
HALI               286
                 ...  
BEAUF               26
BOST                23
BATT                23
ALBA                23
AUDA                20
MARY                18
P400                17
SBD DAUNTLESS       16
MOHAWK              14
BOSTON              14
TBF                 12
B32                 12
LYSANDER   

Some of these aircraft seem to be repeats and inaccurate. Ex- GB24, OB24 and B24 - I suspect these are the same aircraft and erroneously listed as different aircraft. We will first clean up the aircraft names. This can be quite tedious since we need to cross verify with other resources and check if there is an aircraft of the given name. 

By cross checking on wikipedia and google for each of the aircraft types I reached the following conclusions.
1. GB24, OB24, SB24, 0B24, LB30, LIB are different names for B24
2. GB17, OB17, SB17, 0B17 are different names for B17
3. BOST is a different name for A20
4. TBF is a different name for TBF Avenger
5. SBD and SBD Dauntless are different names for A24
6. Bisley is a different name for Blen
7. Mohawk is a different name for P36
8. Auda and Tom could not be identified as aircraft names
9. HVY, LGT and MED refer to Heavy, Light and Medium bombers respectively. These must be further explored based on their nationality.
10. P401 is probably a variant of P40
11. There are some japanese and german aircraft as well such as Hamp, Mary, Val and Ju86. Turns out that Ju86 was used by the South African AF. This can probably be used to get the names of Country if any rows with Ju86 as aircraft type are missing the values for Country column.
12. Some values such as 43 BG, 100SQ refer to the units that operated them. 43 BG operated B17 for most of the war and then operated B24. Since they are only 3 in number we will add them to B17 directly. 35 FG operated P38s. 100SQ operated Lancaster which is a evolved version of Machester. Since there is only one aircraft under 100SQ we will add it to MANC

We will clean the imperfections noticed and get the count again.

In [47]:
data.replace({'Aircraft Series':{'GB24':'B24','OB24':'B24','SB24':'B24','LIB':'B24','0B24':'B24'}},inplace=True)
data.replace({'Aircraft Series':{'BOST':'A20','GB17':'B17','SB17':'B17','OB17':'B17','0B17':'B17'}},inplace=True)
data.replace({'Aircraft Series':{'TBF AVENGER':'TBF','LB30':'B24','SBD DAUNTLESS':'A24','SBD':'A24'}},inplace=True)
data.replace({'Aircraft Series':{'BISLEY':'BLEN','MOHAWK':'P36','P401':'P40','43 BG':'B17'}},inplace=True)
data.replace({'Aircraft Series':{'100 SQ':'MANC','35 FG':'P38','P401':'P40','43 BG':'B17'}},inplace=True)
data.replace({'Aircraft Series':{'AUDA':'UNKNOWN','TOM':'UNKNOWN'}},inplace=True)
data['Aircraft Series'].value_counts()

B17                41973
B24                39377
B25                17359
A20                12584
B26                11980
WELL                6391
HVY                 6298
A26                 4736
P47                 4660
LGT                 3673
P51                 3623
HAMP                3552
BLEN                3224
WHIT                3205
P38                 3002
P40                 2892
B29                 2518
MED                 2379
A36                 1384
P39                  893
STIR                 404
F4U                  361
HALI                 286
HUDSON               180
P61                  174
MANC                 155
VENGEANCE (A31)      118
CATALINA             109
A24                  100
PV-1 VENTURA          89
HURR                  71
F06                   61
P70                   60
TBF                   47
SWORD                 41
BEAUF                 26
UNKNOWN               25
BATT                  23
ALBA                  23
MARY                  18


In [48]:
len(data['Aircraft Series'].unique())

55

In [71]:
data[data['Aircraft Series']=='HVY'].groupby(['Country']).size()

Country
GREAT BRITAIN    6245
USA                48
dtype: int64

The most common british heavy bomber from world war 2 was the halifax. We will replace HVY where Country is UK to HALI.
The most common american heavy bomber from world war 2 was the B17. We will replace HVY where Country is USA to B17.

In [70]:
data[data['Country']=='GREAT BRITAIN'].replace({'Aircraft Series':{'HVY':'HALI'}},inplace=True)
data[data['Country']=='USA'].replace({'Aircraft Series':{'HVY':'B17'}},inplace=True)

In [64]:
data[data['Aircraft Series']=='LGT'].groupby(['Country']).size()

Series([], dtype: int64)

The bombers classified as light are all british. The most common british light bomber in the dataset is the BLEN and hence we will replace LGT with BLEN in the database.

In [65]:
data.replace({'Aircraft Series':{'LGT':'BLEN'}},inplace=True)

In [66]:
data[data['Aircraft Series']=='MED'].groupby(['Country']).size()

Country
GREAT BRITAIN    2366
USA                 5
dtype: int64

The most common british medium bomber was the whitley. Hence all british bombers listed as MED will be replaced by WHIT
The most common american medium bomber was the B25. Hence all american bombers listed as MED will be replaced by B25

In [67]:
data[data['Country']=='GREAT BRITAIN'].replace({'Aircraft Series':{'MED':'WHIT'}},inplace=True)
data[data['Country']=='USA'].replace({'Aircraft Series':{'MED':'B25'}},inplace=True)

Let us print all the aircraft types one last time to ensure all irregularities are fixed.

In [68]:
data['Aircraft Series'].value_counts()

B17                41973
B24                39377
B25                17359
A20                12584
B26                11980
BLEN                6897
WELL                6391
HVY                 6298
A26                 4736
P47                 4660
P51                 3623
HAMP                3552
WHIT                3205
P38                 3002
P40                 2892
B29                 2518
MED                 2379
A36                 1384
P39                  893
STIR                 404
F4U                  361
HALI                 286
HUDSON               180
P61                  174
MANC                 155
VENGEANCE (A31)      118
CATALINA             109
A24                  100
PV-1 VENTURA          89
HURR                  71
F06                   61
P70                   60
TBF                   47
SWORD                 41
BEAUF                 26
UNKNOWN               25
BATT                  23
ALBA                  23
MARY                  18
P400                  17


Next, let us analyze with the Country column. Let us first check the number of rows for each Country.

In [7]:
data.Country.value_counts()

USA              94165
GREAT BRITAIN    31361
NEW ZEALAND        633
AUSTRALIA          316
SOUTH AFRICA        19
Name: Country, dtype: int64

This doesn't give us the rows with null values (in Column 'Country'). Let us check for null values and see if there are any rows with null values.

In [8]:
sum(pd.isnull(data['Country']))

51787

There are about 51787 rows with null values in the dataset. This can definitely be cleaned further and maybe we can even deduce the Country names based on the theatre of war and aircraft used. We will first replace all NAs with NoCountry so it is easier to use the groupby function to observe the data.

In [9]:
data['Country'].fillna('NoCountry',inplace=True)

We will now notice the type of aircraft used and the countries that were involved in different theatres of war.

In [16]:
subdata = data[data['Theater of Operations']=='ETO']
subdata.groupby(['Country','Aircraft Series']).size()

Country        Aircraft Series
GREAT BRITAIN  A20                    2
               A26                    1
               ALBA                   7
               B26                   19
               BATT                  20
               BLEN                2656
               HALI                 263
               HAMP                3542
               HVY                 6125
               LGT                 3657
               LIB                   20
               MANC                 154
               MED                 2323
               STIR                 399
               SWORD                  7
               WELL                5626
               WHIT                3108
NoCountry      0B17                   1
               0B24                   8
               A20                 1088
               A26                  635
               B17                18854
               B24                 9097
               B25                  288
         

In [11]:
data[data['Country']=='NoCountry']

,Mission ID,Mission Date,Theater of Operations,Country,Air Force,Unit ID,Aircraft Series,Mission Type,Takeoff Base,Takeoff Location,...,Incendiary Devices Weight (Tons),Fragmentation Devices,Fragmentation Devices Type,Fragmentation Devices Weight (Pounds),Fragmentation Devices Weight (Tons),Total Weight (Pounds),Total Weight (Tons),Time Over Target,Bomb Damage Assessment,Source ID
13,14,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,95435.0
14,15,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,197541.0
15,16,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,197560.0
16,17,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,95429.0
17,18,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,197535.0
18,19,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,197555.0
19,20,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,95431.0
20,21,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,197537.0
21,22,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,197557.0
22,23,8/15/1943,ETO,NoCountry,NaN,NaN,B17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,95434.0
